In [1]:
import numpy as np
import pandas as pd
import time
from datetime import date, datetime
from OMA_tools.io_data.dates import Dates_Operations
from OMA_tools.ml_models.preprocessing import Preprocessing
#from OMA_tools.ml_models.processing import Forecast_Models
from OMA_tools.ml_models.groups import GROUPS
from OMA_tools.ml_models.postprocessing import Postprocessing

import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger("prophet").setLevel(logging.ERROR)
logging.getLogger("cmdstanpy").setLevel(logging.WARNING)
logging.getLogger('log').setLevel(logging.INFO)
logging.getLogger("cmdstanpy").propagate = False
logging.getLogger("log").propagate = False
logging.getLogger("cmdstanpy").disabled = True

class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

prefix = '/Users/kkozhevnikova/Documents/NSC/Test_groups/data/'
prefix_config = '/Users/kkozhevnikova/Documents/NSC/OMA_tools/ml_models/config.json'
prefix_errors = '/Users/kkozhevnikova/Documents/NSC/Test_groups/Forecast/Errors/'
prefix_full_errors = '/Users/kkozhevnikova/Documents/NSC/Test_groups/Forecast/Results_full/Errors/'
prefix_plots = '/Users/kkozhevnikova/Documents/NSC/Test_groups/Forecast/Plots/'
prefix_full_plots = '/Users/kkozhevnikova/Documents/NSC/Test_groups/Forecast/Results_full/Plots/'

Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.


## Чтение файла с исходными фактическими данными

In [2]:
df = Preprocessing.get_data_for_forecast(filename = f'{prefix}Data_by_months.xlsx',
                  list_of_replacements = ['All 18+', 'All 14-59', 'All 10-45', 'All 14-44',
                      'All 14-54', 'All 25-49', 'All 25-54', 'All 4-45', 'All 6-54', 'W 14-44', 'W 25-59'], 
                                         column_name_with_date = 'Date')

## Определение количества месяцев для прогноза

In [3]:
start_date = pd.to_datetime(df.index[-1], format = '%d.%m.%Y') #нахождение последнего месяца в DataFrame
stop_date = '30.04.2025' # 31 декабря текущего года
forecast_periods = Dates_Operations.diff_month(start_date, stop_date)
forecast_periods

3

In [4]:
test_data = df.iloc[-forecast_periods:]
train_data = df.iloc[:-forecast_periods]

## Функция для расчета

In [5]:
def main(df,
         forecast_periods,
         column_name_with_date, 
         weights_filepath, 
         error_dir = None,
         plots_dir = None, 
         save_dir = None, 
         plots: bool = False, 
         test: bool = False):
    """
        Функция для запуска прогноза ансамбля ML-моделей
        Args:
            filename: Полный путь к файлу с исходными данными
            list_of_replacements: Список из листов, находящихся в файле с исходными данными
            column_name_with_date: Название столбца с датой
            weights_filepath: Полный путь к файлу с весами для каждой модели
            plots_dir: Путь к директории, куда будут сохраняться графики для каждой модели
            save_dir: Путь к директории, куда будут сохраняться итоговые графики с прогнозами
            plots: Переменная типа bool. Если True, то графики строятся. В противном случае нет.
            test: Переменная типа bool. Если True, то тестинг проводится. В противном случае нет.
        Returns:
    """
    #Определение к какой группе относятся данные по тому или иному каналу
    group_1, group_2, group_3, group_4 = GROUPS(df).initiate_group()

    avg_forecasts = []
    
    #GROUP_1
    if not group_1.empty:
        print('', color.BOLD + color.BLUE + 'Результаты работы различных методов для ТВ-каналов с сезонностью и трендом' + color.END, sep = '\n', end = '\n')
        avg_forecast_1 = GROUPS(group_1).process_group(forecast_periods,
                                                       column_name_with_date,
                                                       type_of_group = 'GROUP_1', 
                                                       weights_filepath = weights_filepath, 
                                                       error_dir = error_dir,
                                                       plots_dir = plots_dir, 
                                                       plots = plots, 
                                                       test = test)
        print(color.BOLD + '== ИТОГОВЫЙ РЕЗУЛЬТАТ РАБОТЫ МЕТОДОВ ДЛЯ ТВ-КАНАЛОВ С СЕЗОННОСТЬЮ И ТРЕНДОМ ==' + color.END, avg_forecast_1, sep = '\n', end = '\n')
        if plots:
            Postprocessing(group_1, avg_forecast_1).get_plot(column_name_with_date, 
                                                             f'{save_dir}/Cезонность и тренд')
        avg_forecasts.append(avg_forecast_1)
        
    #GROUP_2
    if not group_2.empty:
        print('', color.BOLD + color.BLUE + 'Результаты работы различных методов для ТВ-каналов с трендом без сезонности' + color.END, sep = '\n', end = '\n')
        avg_forecast_2 = GROUPS(group_2).process_group(forecast_periods,
                                                       column_name_with_date,
                                                       type_of_group = 'GROUP_2', 
                                                       weights_filepath = weights_filepath, 
                                                       error_dir = error_dir,
                                                       plots_dir = plots_dir, 
                                                       plots = plots, 
                                                       test = test)
        print(color.BOLD + '== ИТОГОВЫЙ РЕЗУЛЬТАТ РАБОТЫ МЕТОДОВ ДЛЯ ТВ-КАНАЛОВ С ТРЕНДОМ БЕЗ СЕЗОННОСТИ ==' + color.END, avg_forecast_2, sep = '\n', end = '\n')
        if plots:
            Postprocessing(group_2, avg_forecast_2).get_plot(column_name_with_date, 
                                                             f'{save_dir}/Тренд без сезонности')
        avg_forecasts.append(avg_forecast_2)

    #GROUP_3
    if not group_3.empty:
        print('', color.BOLD + color.BLUE + 'Результаты работы различных методов для ТВ-каналов с сезонностью без тренда' + color.END, sep = '\n', end = '\n')
        avg_forecast_3 = GROUPS(group_3).process_group(forecast_periods,
                                                       column_name_with_date,
                                                       type_of_group = 'GROUP_3', 
                                                       weights_filepath = weights_filepath, 
                                                       error_dir = error_dir,
                                                       plots_dir = plots_dir, 
                                                       plots = plots, 
                                                       test = test)
        print(color.BOLD + '== ИТОГОВЫЙ РЕЗУЛЬТАТ РАБОТЫ МЕТОДОВ ДЛЯ ТВ-КАНАЛОВ С СЕЗОННОСТЬЮ БЕЗ ТРЕНДА ==' + color.END, avg_forecast_3, sep = '\n', end = '\n')
        if plots:
            Postprocessing(group_3, avg_forecast_3).get_plot(column_name_with_date, 
                                                             f'{save_dir}/Сезонность без тренда')
        avg_forecasts.append(avg_forecast_3)

    #GROUP_4
    if not group_4.empty:
        print('', color.BOLD + color.BLUE + 'Результаты работы различных методов для ТВ-каналов без сезонности и без тренда'+ color.END, sep = '\n', end = '\n')
        avg_forecast_4 = GROUPS(group_4).process_group(forecast_periods,
                                                       column_name_with_date,
                                                       type_of_group = 'GROUP_4', 
                                                       weights_filepath = weights_filepath, 
                                                       error_dir = error_dir,
                                                       plots_dir = plots_dir, 
                                                       plots = plots, 
                                                       test = test)
        print(color.BOLD + '== ИТОГОВЫЙ РЕЗУЛЬТАТ РАБОТЫ МЕТОДОВ ДЛЯ ТВ-КАНАЛОВ БЕЗ СЕЗОННОСТИ И БЕЗ ТРЕНДА ==' + color.END, avg_forecast_4, sep = '\n', end ='\n')
        if plots:
            Postprocessing(group_4, avg_forecast_4).get_plot(column_name_with_date, 
                                                             f'{save_dir}/Без сезонности и без тренда')
        avg_forecasts.append(avg_forecast_4)
    general_df = Postprocessing.ensemble_of_models(df, *avg_forecasts)
    return general_df

# **Расчет**

In [ ]:
start = time.perf_counter()

general_df = main(df = train_data,
                  forecast_periods = forecast_periods, 
                  column_name_with_date = 'Date', 
                  weights_filepath = prefix_config)
end = time.perf_counter()

print(f'ВРЕМЯ РАБОТЫ КОДА СОСТАВИЛО: {((end-start) / 60):0.2f} мин.')

INFO - (NP.config.__post_init__) - Note: Trend changepoint regularization is experimental.
INFO - (NP.config.__post_init__) - Note: Trend changepoint regularization is experimental.
INFO - (NP.config.__post_init__) - Note: Trend changepoint regularization is experimental.
INFO - (NP.config.__post_init__) - Note: Trend changepoint regularization is experimental.
INFO - (NP.config.__post_init__) - Note: Trend changepoint regularization is experimental.
INFO - (NP.config.__post_init__) - Note: Trend changepoint regularization is experimental.
INFO - (NP.config.__post_init__) - Note: Trend changepoint regularization is experimental.
INFO - (NP.config.__post_init__) - Note: Trend changepoint regularization is experimental.
INFO - (NP.config.__post_init__) - Note: Trend changepoint regularization is experimental.
INFO - (NP.config.__post_init__) - Note: Trend changepoint regularization is experimental.
INFO - (NP.config.__post_init__) - Note: Trend changepoint regularization is experimental.


Результаты работы различных методов для ТВ-каналов с сезонностью и трендом
РЕЗУЛЬТАТ РАБОТЫ ФУНКЦИИ DEC_WITH_TREND_PERIODS
РЕЗУЛЬТАТ РАБОТЫ ФУНКЦИИ SEASONDEC_PERIODS
            ЗВЕЗДА (НОВОСИБИРСК) All 18+  \
2024-11-01                        4.4545   
2024-12-01                        4.2243   
2025-01-01                        4.7075   

            ПЕРВЫЙ КАНАЛ (НИЖНИЙ НОВГОРОД) All 18+  \
2024-11-01                                  6.6268   
2024-12-01                                  6.8260   
2025-01-01                                  6.6194   

            РЕН ТВ (ЕКАТЕРИНБУРГ) All 18+  РОССИЯ 1 (НИЖНИЙ НОВГОРОД) All 18+  \
2024-11-01                         8.8561                             16.4482   
2024-12-01                         8.8005                             19.2074   
2025-01-01                         9.4816                             18.9014   

            РОССИЯ 1 (ЧЕЛЯБИНСК) All 18+  РОССИЯ 24 (УФА) All 18+  \
2024-11-01                       24.4363     

INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to [97.826]% of the data.
INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to [97.826]% of the data.
INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to [97.826]% of the data.
INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to [97.826]% of the data.
INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to [97.826]% of the data.
INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to [97.826]% of the data.
INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to [97.826]% of the data.
INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to [97.826]% of the data.
INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to [97.826]% of the data.
INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to [97.826]% of the data.
INFO - (NP.df_utils._infer_fre

Training: |                                               | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

...

Training: |                                               | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 400


Training: |                                               | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

In [ ]:
general_df

# Вывод ошибок прогноза

In [ ]:
general_MAPE = (((general_df / test_data) - 1) * 100).abs()
general_MAPE = general_MAPE.reset_index()
general_MAPE = general_MAPE.rename(columns = {general_MAPE.columns[0]: 'Date'})
general_MAPE.set_index('Date', inplace = True)
mean_values = general_MAPE.mean()
# Добавляем новую строку с названием индекса "Mean"
general_MAPE.loc['Mean'] = mean_values
general_MAPE.to_excel(f'{prefix_full_errors}forecast_error_4.xlsx')

In [9]:
general_df = general_df.reset_index()
general_df.to_excel(f'{prefix_output}forecast_output.xlsx')